In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import math
import os
import pickle
import re

In [2]:
ACTION_FILE = './datafile/JData_Action.csv'
COMMENT_FILE = './datafile/JData_Comment.csv'
PRODUCT_FILE = './datafile/JData_Product.csv'
USER_FILE = './datafile/JData_User.csv'

In [2]:
# with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
#     user_load_buy_frame = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-sku_id-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id','rb') as f:
#     user_sku_info = pickle.load(f)
# comment_dataset = pd.read_csv(COMMENT_FILE)
# product_dataset = pd.read_csv(PRODUCT_FILE)

# b= pd.read_csv('pre_5_22.csv')
# b = pd.DataFrame(b['user_id'])
# test_data = pd.merge(b,a,on='user_id',how='left')
# test_data = pd.DataFrame(test_data[test_data.sku_id.isnull()]['user_id'])
# test_data = pd.merge(test_data,user_dataset,on='user_id',how='left')
# test_data = test_data[test_data['cate']==8][['user_id','sku_id']]
# a = pd.read_csv('pre_5_22 - 副本.csv')
# a = a.sort_values(by='user_id',ascending=True)
# a.to_csv('pre_4_22_2.csv',index=False)

In [7]:
#选择登陆天数小于7天的用户,并且最后7天中必须登录2到3天,又小于5天
def get_load_less_7():
    dump_path = './cache/Action_2016-02-01_2016-04-16_load_less_7'
    if os.path.exists(dump_path):
        with open(dump_path,'rb') as f:
            df = pickle.load(f)
    else:
        with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
            user_load_buy_frame = pickle.load(f)
        df = user_load_buy_frame[user_load_buy_frame['load_days']<=7][['user_id','2016-04-09','2016-04-10','2016-04-11','2016-04-12','2016-04-13','2016-04-14','2016-04-15']]
        df['last_week_load_days'] = df.sum(axis=1) - df['user_id']
        df = df[(df['last_week_load_days']>=1)&(df['last_week_load_days']<=5)][['user_id','last_week_load_days']]
        df = pd.merge(df,user_load_buy_frame,on='user_id',how='left')
        df = df.drop(['2016-02-01','2016-02-02','2016-02-03','2016-02-04','2016-02-05','2016-02-06','2016-02-07',\
                      '2016-02-08','2016-02-09','2016-02-10','2016-02-11','2016-02-12','2016-02-13','2016-02-14','2016-02-15',\
                      '2016-02-16','2016-02-17','2016-02-18','2016-02-19','2016-02-20','2016-02-21','2016-02-22','2016-02-23',\
                      '2016-02-24','2016-02-25','2016-02-26','2016-02-27','2016-02-28','2016-02-29','2016-03-01',\
                      '2016-03-02','2016-03-03','2016-03-04','2016-03-05','2016-03-06','2016-03-07','2016-03-08','2016-03-09',\
                      '2016-03-10','2016-03-11','2016-03-12','2016-03-13','2016-03-14','2016-03-15','2016-03-16','2016-03-17',\
                      '2016-03-18','2016-03-19','2016-03-20','2016-03-21','2016-03-22','2016-03-23','2016-03-24','2016-03-25',\
                      '2016-03-26','2016-03-27','2016-03-28','2016-03-29','2016-03-30','2016-03-31',\
                      'cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'],axis=1)
        df = df[(df['last_buy_cate']!=8)]
        df = df[df['cate_8']>0]
        df = df[df['last_buy_time']<'2016-03-01 00:00:00']
        with open(dump_path,'wb') as f:
                pickle.dump(df,f)
    return df

In [1]:
# b = get_load_less_7()
# df = pd.DataFrame(b['user_id'])
# df = pd.merge(df,user_dataset,on='user_id',how='left')
# df = df[df['cate']==8][['user_id','time']]
# #新加上去的
# df = df[df['time']>'2016-04-09 00:00:00']
# df['time0'] = df.time.apply(lambda x :int(x[5:7]))
# df['time1'] = df.time.apply(lambda x :int(x[8:10]))
# df['time2'] = df.time.apply(lambda x :int(x[11:13]))
# df['time3'] = df.time.apply(lambda x :int(x[14:16]))
# df['time_point'] = df['time0']*30*24*60+df['time1']*24*60+df['time2']*60+df['time3']
# df = df[['user_id','time_point','time','time2']]
# # df.groupby(['user_id','time_point'],as_index=False).count()
# df1 = df.groupby(['user_id'],as_index=False).time_point.max()
# df2 = df.groupby(['user_id'],as_index=False).time_point.min()
# df1.columns = ['user_id','max_time']
# df2.columns = ['user_id','min_time']
# df3 = pd.merge(df1,df2,on='user_id',how='left')
# df3['see_longtime'] = df3['max_time'] - df3['min_time']
# df = pd.merge(df,df3,on='user_id',how='left')
# df = pd.merge(df,type2,on='user_id',how='left')
# not_buy_user = pd.DataFrame(df[(df['see_longtime']<=10)&(df['time']<'2016-04-13 00:00:00')&(df['type_2']==0)&(df['type_3']==0)&(df['type_5']==0)]\
#              ['user_id']).drop_duplicates()    #432    173
# not_buy_user['label'] = 0
# df4 = pd.merge(df,not_buy_user,on='user_id',how='left')
# df4 = df4[df4.label!=0][['user_id','see_longtime','time2','time','type_1','type_2','type_3','type_5','type_6','cate_8_num']]     # 6400条左右
# pd.DataFrame(df4[df4['see_longtime']<=10]['user_id']).drop_duplicates()                  #1984条
# df4[(df4['see_longtime']<=5)&(df4['type_2']>0)]
# df4['ratio'] = df4['cate_8_num'] / df4['see_longtime']

# df4 = pd.merge(df4,b,on='user_id',how='left')
# df4 = df4.drop(['last_buy_cate','last_buy_cate_8_time'],axis=1)

# ###将所有在22-24，0-2,11-13，浏览的用户保存下来
# retain_user_dataset_1 = df4[(df4['time2']>=22)]
# retain_user_dataset_2 =df4[(df4['time2']>=11)&(df4['time2']<=13)]
# retain_user_dataset_3 = df4[(df4['time2']<=2)]
# retain_user_dataset = pd.concat([retain_user_dataset_1,retain_user_dataset_2,retain_user_dataset_3],axis=0)
# pd.DataFrame(retain_user_dataset['user_id']).drop_duplicates()                 #3602

# temp_retain = retain_user_dataset[(retain_user_dataset['type_2']>0)&(retain_user_dataset['type_5']>0)]
# temp_retain = temp_retain.drop(['time2','time'],axis=1).drop_duplicates()
# pd.DataFrame(temp_retain[temp_retain['time']>'2016-04-13 00:00:00']['user_id']).drop_duplicates()    #104
# temp_retain[temp_retain['last_week_load_days']==3]   #23   31   45

# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-sku_id-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# comment_dataset = pd.read_csv(COMMENT_FILE)
# product_dataset = pd.read_csv(PRODUCT_FILE)
# pre_user_data = temp_retain
# pre_user_id_data = pd.DataFrame(pre_user_data['user_id'])
# pre_user_id_data = pd.merge(pre_user_id_data,user_dataset,on='user_id',how='left')
# pre_user_id_data = pre_user_id_data[pre_user_id_data['cate']==8][['user_id','sku_id']]

# pre_user_id_data_1 = pd.merge(pre_user_id_data,user_sku_info,on=('user_id','sku_id'),how='left')
# pre_user_id_data_1

In [47]:
# temp = clean_data()
# temp1 = temp[['user_id','type']]
# type_dummies = pd.get_dummies(temp1['type'],prefix='type')
# type2 = pd.concat([temp1,type_dummies],axis=1)
# type2 = type2.drop(['cate','time','type'])
# type2 = type2.groupby(['user_id'],as_index=False).sum()
# type2['cate_8_num'] = type2.sum(axis=1) - type2['user_id']
type2

,user_id,type_1,type_2,type_3,type_5,type_6,cate_8_num
0,200005.0,14.0,0.0,0.0,0.0,22.0,36.0
1,200054.0,60.0,0.0,0.0,0.0,69.0,129.0
2,200055.0,6.0,0.0,0.0,0.0,7.0,13.0
3,200083.0,18.0,0.0,0.0,0.0,34.0,52.0
4,200088.0,62.0,0.0,0.0,0.0,114.0,176.0
5,200092.0,4.0,0.0,0.0,0.0,4.0,8.0
6,200117.0,3.0,0.0,0.0,0.0,9.0,12.0
7,200128.0,2.0,0.0,0.0,0.0,9.0,11.0
8,200141.0,18.0,0.0,0.0,0.0,22.0,40.0
9,200160.0,4.0,0.0,0.0,0.0,2.0,6.0


In [116]:
# a = pd.read_csv('pre_5_25_1.csv')
# d = pd.read_csv('pre_5_22.csv')
# d = pd.DataFrame(d['user_id'])
# d = pd.merge(d,a,on='user_id',how='left')
# b = pd.read_csv('pre_5_25.csv')
# b = pd.DataFrame(b['user_id']).drop_duplicates()
# c= pd.merge(b,a,on='user_id',how='left')
# c
# a= a.sort_values(by='proba',ascending=False)
# a.iloc[:1000]
# d[d['proba']>=0.089]

,user_id,proba
0,200208,0.157169
1,200288,0.257368
2,200690,0.102714
3,200756,0.196890
4,201123,0.184755
5,201385,0.206888
6,201568,0.554301
7,202469,0.108600
10,203534,0.565907
11,203624,0.218739


In [126]:
# with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
#     user_load_buy_frame = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-sku_id-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# comment_dataset = pd.read_csv(COMMENT_FILE)
# product_dataset = pd.read_csv(PRODUCT_FILE)
# ##引入商品信息
# pre_user_data = temp_retain
# pre_user_id_data = pd.DataFrame(pre_user_data['user_id'])
# pre_user_id_data = pd.merge(pre_user_id_data,user_dataset,on='user_id',how='left')
# pre_user_id_data = pre_user_id_data[pre_user_id_data['cate']==8]
# pre_user_id_data_1 = pd.merge(pre_user_id_data,comment_dataset,on='sku_id',how='left')
# pre_user_id_data_1 = pd.merge(pre_user_id_data_1,product_dataset,on='sku_id',how='left')
# pre_user_id_data_1 = pre_user_id_data_1.drop(['dt','a1','a2','a3'],axis=1)




# temp_1 = pre_user_id_data_1[(pre_user_id_data_1['type']==2)&(pre_user_id_data_1['type']==3)&(pre_user_id_data_1['type']==5)].drop_duplicates()
# temp_2 = temp_1[['user_id','sku_id','type','comment_num','has_bad_comment','bad_comment_rate']]
# pre_user_id_data_1
# temp_2 = temp_2.groupby(['user_id','sku_id','type','comment_num','has_bad_comment'],as_index=False).bad_comment_rate.min()
# delta_sku_id = temp_2[temp_2['type']==3][['user_id','sku_id']]
# delta_sku_id['label'] = 0
# retain_data = pd.merge(temp_2,delta_sku_id,on=('user_id','sku_id'),how='left')
# retain_data = retain_data[retain_data['label']!=0]
# retain_data = temp_2
# retain_data['label'] = np.nan
# temp_3 = retain_data.groupby(['user_id'],as_index=False).bad_comment_rate.min()
# temp_3['first'] = 1
# retain_data = pd.merge(retain_data,temp_3,on=('user_id','bad_comment_rate'),how='left')
# temp_4 = retain_data.groupby(['user_id'],as_index=False).bad_comment_rate.max()
# temp_4['last'] = 3
# retain_data = pd.merge(retain_data,temp_4,on=('user_id','bad_comment_rate'),how='left')
# retain_data_1 = retain_data[(retain_data['first']==1)&(retain_data['last']==3)][['user_id','sku_id','label']]
# a = retain_data_1.fillna(1)
# retain_data_2 = retain_data[(retain_data['first']!=1)&(retain_data['last']==3)][['user_id','sku_id','label']]
# b = retain_data_2.fillna(3)
# retain_data_3 = retain_data[(retain_data['first']!=1)&(retain_data['last']!=3)][['user_id','sku_id','label']]
# c = retain_data_3.fillna(2)
# retain_data = pd.concat([a,b,c],axis=0)
# retain_data
# d = retain_data




,cate,sku_id,type,user_id
0,10,107774,1,272629.0
1,10,107774,1,272629.0
2,10,107774,6,272629.0
3,10,107774,1,272629.0
4,10,107774,6,272629.0
5,10,107774,6,272629.0
6,10,107774,1,272629.0
7,9,87598,1,284046.0
8,6,153817,3,208720.0
9,9,87598,1,284046.0


# 5.22

In [5]:
# with open('./cache/Action_2016-04-09_2016-04-16_with_cate-sku_id-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# # pre_data = pd.read_csv('pre_5_20_3.csv')
# pre_user_id_data = pd.DataFrame(pre_data['user_id'])
# pre_user_id_data = pd.merge(pre_user_id_data,user_dataset,how='left',on='user_id')
# pre_user_id_data_1 = pre_user_id_data[pre_user_id_data['cate']==8]
# pre_user_id_data_2 = pre_user_id_data_1[['user_id','sku_id','type']]
# pre_1 = pre_user_id_data_1.groupby(['user_id','sku_id','type'],as_index = False).count()
# pre_2 = pre_user_id_data_2.groupby(['user_id','sku_id'],as_index=False).count()
# pre_2.columns = ['user_id','sku_id','number']
# temp_2 = pre_2.groupby(['user_id'],as_index=False).number.max()
# pre_3 = pd.merge(temp_2,pre_2,on=['user_id','number'],how='left')[['user_id','sku_id']]
#引进时间变量
# with open('./cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id','rb') as f:
#     user_browse_time_dataset = pickle.load(f)
# pre_4 = pd.merge(pre_3,user_browse_time_dataset,how='left',on=['user_id','sku_id'])[['user_id','sku_id','time']]
# pre_5 = pre_4.groupby(['user_id'],as_index=False).time.max()
# pre_5 = pd.merge(pre_5,pre_4,on=['user_id','time'],how='left')[['user_id','sku_id']].drop-duplicates()
# a = pre_5.drop_duplicates()
# a.to_csv('pre_5_22.csv',index=False)

#引进商品优先级
# a = pd.read_csv('pre_5_22.csv')


,sku_id,time,type,user_id
0,107774,2016-02-01 00:00:00,1,272629.0
1,107774,2016-02-01 00:00:00,1,272629.0
2,107774,2016-02-01 00:00:00,6,272629.0
3,107774,2016-02-01 00:00:00,1,272629.0
4,107774,2016-02-01 00:00:00,6,272629.0
5,107774,2016-02-01 00:00:00,6,272629.0
6,107774,2016-02-01 00:00:00,1,272629.0
7,87598,2016-02-01 00:00:01,1,284046.0
8,153817,2016-02-01 00:00:01,3,208720.0
9,87598,2016-02-01 00:00:01,1,284046.0


In [45]:
temp_1 = pd.merge(pre_3,pre_1,on=['user_id','sku_id'],how='left')
temp_1[temp_1['type']==2]

,user_id,sku_id,type,cate
3,200288,166707,2,1
8,200756,57018,2,1
11,201123,84389,2,1
14,201123,133436,2,1
19,201568,12564,2,1
30,202845,47895,2,1
38,203624,62351,2,1
41,203728,44854,2,1
46,204399,44854,2,1
49,204535,154636,2,1


# 5.21

In [21]:
# a = pd.read_csv('pre_5_19_1.csv')
# a = pd.DataFrame(a['user_id'])
# b = pd.read_csv('pre_5_20_1.csv')
# b_1 = pd.DataFrame(b['user_id'])
# c = pd.concat([a,b_1],axis=0).drop_duplicates()
# d = pd.merge(c,b,on='user_id',how='left')
# e = d.fillna(range(176))
# e = e.astype(int)
# e.to_csv('pre_5_20_3.csv',index=False)
# b = b.astype(int)
# b.to_csv('pre_5_20_1.csv',index=False)
# e

ValueError: invalid fill value with a <class 'range'>

# 5.19号

In [ ]:
# with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
#     user_load_buy_frame = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# user_info_dataset = pd.read_csv(USER_FILE,encoding='gbk')
# user_load_buy_frame = user_load_buy_frame.drop(['cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'],axis=1)
# user_load_buy_frame = user_load_buy_frame[user_load_buy_frame['last_buy_time']!=user_load_buy_frame['last_buy_cate_8_time']]
# user_load_buy_frame_1 = user_load_buy_frame[user_load_buy_frame['last_buy_time']=='2016-01-30 00:00:00']
# just_last_three_day_load = user_load_buy_frame_1[(user_load_buy_frame_1['load_days']<=3)]
# just_last_three_day_load = just_last_three_day_load[(user_load_buy_frame_1['2016-04-15']==1)|\
#             (user_load_buy_frame_1['2016-04-14']==1)|(user_load_buy_frame_1['2016-04-13']==1)]
# just_last_three_day_load_user = just_last_three_day_load[['user_id','cate_8','browse_count','cate_8_ratio']]
# just_last_three_day_load_user = pd.merge(just_last_three_day_load_user,user_dataset,on='user_id',how='left')
# a = just_last_three_day_load_user[(just_last_three_day_load_user['cate']==8)&(just_last_three_day_load_user['time']>'2016-04-13 00:00:00')]
#                     #  总共对了216条，
# c = pd.DataFrame(a[(a['type']==2)|(a['type']==3)|(a['type']==5)]['user_id']).drop_duplicates()
# d = pd.read_csv('pre_5_19_1.csv')
# b = pd.DataFrame(a['user_id']).drop_duplicates()
# e = pd.merge(b,d,on='user_id',how='left')
# f = e.fillna('2016-04-01')
# f = f[f['sku_id']=='2016-04-01']
# g = pd.DataFrame(f['user_id'])
#查找所有只看了第八类商品的用户
# browse_cate_8 = pd.merge(g,user_load_buy_frame,on='user_id',how='left')
# m = pd.DataFrame(browse_cate_8[(browse_cate_8['cate_8_ratio']>0.5)&(browse_cate_8['cate_8']>20)]['user_id']).drop_duplicates()
# l = pd.DataFrame(browse_cate_8[(browse_cate_8['cate_8_ratio']==1)&(browse_cate_8['cate_8']>49)]['user_id']).drop_duplicates()

# low_ratio_user_action_data = pd.merge(g,user_dataset,on='user_id',how='left')
# type_dummies = pd.get_dummies(low_ratio_user_action_data['type'],prefix='Action')
# type_dummies[type_dummies['Action_2']==1]


# low_ratio_user_action_data_1 =low_ratio_user_action_data[(low_ratio_user_action_data['cate']==8)&\
#                                                          (low_ratio_user_action_data['time']>'2016-04-09 00:00:00')]
# h = pd.DataFrame(low_ratio_user_action_data_1[(low_ratio_user_action_data_1['time']>'2016-04-09 00:00:00')&\
#                              (low_ratio_user_action_data_1['type']==2)]
# low_ratio_user_action_data_1[low_ratio_user_action_data_1['time']>'2016-04-14 00:00:00']
# low_ratio_user_action_data_1.groupby(['user_id','cate','type'],as_index=False).time.max()
# n = pd.DataFrame(low_ratio_user_action_data_1[(low_ratio_user_action_data_1['time']>'2016-04-09 00:00:00')&\
#                              ((low_ratio_user_action_data_1['type']==2)|(low_ratio_user_action_data_1['type']==3)|\
#                               (low_ratio_user_action_data_1['type']==5))]['user_id']).drop_duplicates()
# low_ratio_user_action_data_2 = low_ratio_user_action_data[(low_ratio_user_action_data['time']>'2016-04-15 20:00:00')&\
#                                                           (low_ratio_user_action_data['cate']==8)]
# s = pd.DataFrame(low_ratio_user_action_data_2['user_id']).drop_duplicates()
# p = pd.concat([h,n,s],axis=0).drop_duplicates()



#测试开始
# user_id = pd.read_csv('pre_5_19_1.csv')['user_id']
# user_id = pd.DataFrame(user_id)
# user_id = pd.concat([p,user_id],axis=0).drop_duplicates()
# user_sku_id = pd.merge(user_id,user_dataset,on='user_id',how='left')
# user_sku_id = user_sku_id[user_sku_id['cate']==8]
# user_sku_id_1 = user_sku_id[(user_sku_id['type']==2)|(user_sku_id['type']==3)|(user_sku_id['type']==5)]
# user_sku_id_2 = user_sku_id_1.groupby(['user_id'],as_index=False).time.max()
# with open('./cache/Action_2016-02-01_2016-04-16_with_sku_id-time-type-user_id','rb') as f:
#     sku_id_dataset = pickle.load(f)
# user_sku_id_2.columns = ['user_id','cart_time']
# user_sku_id_3 = pd.merge(user_sku_id_2,sku_id_dataset,on='user_id',how='left')
# user_sku_id_3 = user_sku_id_3[(user_sku_id_3['time']==user_sku_id_3['cart_time'])]
# user_sku_id_3 = user_sku_id_3[(user_sku_id_3['type']==2)|(user_sku_id_3['type']==5)]
# protect_data = user_sku_id_3[['user_id','sku_id']].drop_duplicates()
# protect_data.to_csv('pre_5_20_1.csv',index=False)

#点击次数最多的
# lost_user_id = pd.merge(user_id,protect_data,on='user_id',how='left')
# lost_user_id = lost_user_id.fillna('2016-10-10')
# lost_user_id = pd.DataFrame(lost_user_id[lost_user_id['sku_id']=='2016-10-10']['user_id'])
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-sku_id-type-user_id','rb') as f:
#     user_sku_cate_dataset = pickle.load(f)
# lost_user_id = pd.merge(lost_user_id,user_sku_cate_dataset,on='user_id',how='left')
# lost_user_id
# lost_user_id_1 = lost_user_id[lost_user_id['cate']==8][['user_id','sku_id']]
# lost_user_id_1.groupby(['user_id','sku_id'],as_index=False).count()

In [124]:
lost_user_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 588 entries, 0 to 587
Data columns (total 2 columns):
user_id    588 non-null float64
sku_id     334 non-null float64
dtypes: float64(2)
memory usage: 13.8 KB


# 5.18,在最后一次购买商品不是第8类的用户中寻找

In [45]:
# with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
#     user_load_buy_frame = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# user_info_dataset = pd.read_csv(USER_FILE,encoding='gbk')
# user_load_buy_frame = user_load_buy_frame.drop(['cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'],axis=1)
# user_load_buy_frame = user_load_buy_frame[user_load_buy_frame['last_buy_time']!=user_load_buy_frame['last_buy_cate_8_time']]
# user_load_buy_frame_1 = user_load_buy_frame[user_load_buy_frame['last_buy_cate_8_time'].notnull()]
# user_browse_cate_8 = user_load_buy_frame_1[user_load_buy_frame_1['cate_8']>0]
# temp_1 = user_browse_cate_8[(user_browse_cate_8['2016-04-09']==1)|(user_browse_cate_8['2016-04-10']==1)|\
#                             (user_browse_cate_8['2016-04-11']==1)|(user_browse_cate_8['2016-04-12']==1)|\
#                            (user_browse_cate_8['2016-04-13']==1)|(user_browse_cate_8['2016-04-14']==1)|\
#                            (user_browse_cate_8['2016-04-15']==1)]                      #最后一个礼拜必须登录了
# temp_1.iloc[:,-14:]




#所有在这两个月买过第8类商品的用户在4.16号到4.20都不曾买过第八类商品


# 最后几天登陆了一天，且看了第8类商品的用户

In [87]:
# user_load_buy_frame_2 = user_load_buy_frame[user_load_buy_frame['last_buy_time']=='2016-01-30 00:00:00']
# just_last_day_load = user_load_buy_frame_2[(user_load_buy_frame_2['load_days']==1)&(user_load_buy_frame_2['2016-04-15']==1)]
# just_last_day_load_user = just_last_day_load[['user_id','cate_8','browse_count','cate_8_ratio']]
# just_last_day_load_user = pd.merge(just_last_day_load_user,user_dataset,on='user_id',how='left')
# just_last_day_load_user


#
# just_last_secend_day_load = user_load_buy_frame_2[(user_load_buy_frame_2['load_days']==1)&(user_load_buy_frame_2['2016-04-14']==1)]
# just_last_secend_day_load_user = just_last_secend_day_load[['user_id','cate_8','browse_count','cate_8_ratio']]
# just_last_secend_day_load_user = pd.merge(just_last_secend_day_load_user,user_dataset,on='user_id',how='left')
# just_last_secend_day_load_user


#
# just_last_third_day_load = user_load_buy_frame_2[(user_load_buy_frame_2['load_days']==1)&(user_load_buy_frame_2['2016-04-13']==1)]
# just_last_third_day_load_user = just_last_third_day_load[['user_id','cate_8','browse_count','cate_8_ratio']]
# just_last_third_day_load_user = pd.merge(just_last_third_day_load_user,user_dataset,on='user_id',how='left')
# just_last_third_day_load_user


# #最后三天登录了，且查看了第8类商品的用户
# just_last_three_day_load = user_load_buy_frame_2[(user_load_buy_frame_2['load_days']<=3)]
# just_last_three_day_load = just_last_three_day_load[(user_load_buy_frame_2['2016-04-15']==1)|(user_load_buy_frame_2['2016-04-14']==1)|(user_load_buy_frame_2['2016-04-13']==1)]
# just_last_three_day_load_user = just_last_three_day_load[['user_id','cate_8','browse_count','cate_8_ratio']]
# just_last_three_day_load_user = pd.merge(just_last_three_day_load_user,user_dataset,on='user_id',how='left')
# a = just_last_three_day_load_user[(just_last_three_day_load_user['cate']==8)&(just_last_three_day_load_user['time']>'2016-04-13 00:00:00')]
#总共对了216条，
# c = pd.DataFrame(a[(a['type']==2)|(a['type']==3)|(a['type']==5)]['user_id']).drop_duplicates()
# d = pd.concat([just_last_day_load_user,just_last_secend_day_load_user,just_last_third_day_load_user,c],axis=0)['user_id']
# d = pd.DataFrame(d).drop_duplicates()
d.drop_duplicates()

,user_id,sku_id
0,200288.0,0
140,200690.0,1
145,201568.0,2
148,203534.0,3
275,204642.0,4
283,206995.0,5
318,207468.0,6
385,207483.0,7
409,207493.0,8
411,207494.0,9


In [67]:
just_last_three_day_load.describe()

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,cate_8,browse_count,cate_8_ratio
count,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,...,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000,1158.000000
mean,251796.848877,0.007772,0.006045,0.005181,0.004318,0.006908,0.004318,0.002591,0.000864,0.005181,...,0.058722,0.071675,0.094128,0.292746,0.417098,0.491364,2.337651,49.623489,68.942142,0.794365
std,30885.963145,0.087854,0.077547,0.071826,0.065596,0.082865,0.065596,0.050855,0.029386,0.071826,...,0.235205,0.258061,0.292133,0.455219,0.493292,0.500141,0.759258,63.776741,87.001918,0.303261
min,200208.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
25%,224813.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,12.000000,19.000000,0.613690
50%,252367.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,30.000000,42.000000,1.000000
75%,278602.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,3.000000,62.000000,82.750000,1.000000
max,305287.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.000000,749.000000,822.000000,1.000000


In [88]:
# pd.DataFrame(just_last_three_day_load_user['user_id']).drop_duplicates()
# temp = pd.DataFrame(a['user_id']).drop_duplicates()            #1049
# temp['sku_id'] = [x for x in range(len(temp))]
# temp = temp.astype(int)
# temp.to_csv('pre_5_18_2.csv',index=False)
# pd.DataFrame(b['user_id']).drop_duplicates()
d['sku_id'] = [x for x in range(len(d))]
d = d.astype(int)
d.to_csv('pre_5_19_1.csv',index=False)

In [89]:
d

,user_id,sku_id
0,200288,0
140,200690,1
145,201568,2
148,203534,3
275,204642,4
283,206995,5
318,207468,6
385,207483,7
409,207493,8
411,207494,9


In [62]:
just_last_third_day_load

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,last_buy_cate,last_buy_cate_8_time,cate_8,browse_count,cate_8_ratio
3490,205514.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,7.0,19.0,0.368421
3532,205574.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,0.0,6.0,0.000000
3900,206161.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,23.0,23.0,1.000000
4802,207390.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,1.0,1.0,1.000000
4803,207391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,5.0,5.0,1.000000
16048,219266.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,6.0,104.0,0.057692
20687,224213.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,6.0,6.0,1.000000
22830,226457.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,10.0,10.0,1.000000
26434,230274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,5.0,5.0,1.000000
29240,233233.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,2016-01-30 00:00:00,NaN,NaN,14.0,14.0,1.000000


In [60]:
just_last_secend_day_load_user[(just_last_secend_day_load_user['type']==5)|(just_last_secend_day_load_user['type']==2)|(just_last_secend_day_load_user['type']==3)]

,user_id,cate_8,browse_count,cate_8_ratio,cate,time,type
11,202469.0,8.0,8.0,1.000000,8,2016-04-14 23:26:19,5
96,204535.0,14.0,14.0,1.000000,8,2016-04-14 17:46:42,2
125,207427.0,106.0,391.0,0.271100,8,2016-04-14 12:29:57,2
237,207427.0,106.0,391.0,0.271100,7,2016-04-14 14:07:13,2
403,207427.0,106.0,391.0,0.271100,7,2016-04-14 14:22:32,2
528,210190.0,7.0,36.0,0.194444,5,2016-04-14 16:32:31,2
550,210801.0,28.0,28.0,1.000000,8,2016-04-14 07:22:21,2
554,210801.0,28.0,28.0,1.000000,8,2016-04-14 07:22:52,2
642,218090.0,37.0,114.0,0.324561,5,2016-04-14 11:26:51,2
667,218090.0,37.0,114.0,0.324561,4,2016-04-14 12:12:46,2


# 下面提交的633条数据没一条是对的

In [44]:
#查看浏览了第8类商品的用户行为
# user_id = user_browse_cate_8[['user_id','last_buy_cate_8_time','last_buy_time']]
# user_id_action = pd.merge(user_id,user_dataset,on='user_id',how='left')
# last_15_days = user_id_action[(user_id_action['cate']==8)&(user_id_action['time']>user_id_action['last_buy_time'])]
# last_15_days = last_15_days[last_15_days['time']>'2016-04-01 00:00:00']
# last_15_days_1 = last_15_days.drop_duplicates()
# a = pd.DataFrame(last_15_days_1[last_15_days_1['type']==2]['user_id']).drop_duplicates()   #131
# b = pd.DataFrame(last_15_days_1['user_id']).drop_duplicates()
# temp_b = pd.merge(b,temp_1,on='user_id',how='left')
# temp_b.dropna(axis=0)
# temp_a = pd.merge(a,temp_1,on='user_id',how='left')
# temp_a.dropna(axis=0)
# temp_b_2 = pd.merge(b,user_browse_cate_8,on='user_id',how='left')   #mean cate_8 67
# temp_b_2[(temp_b_2['cate_8_ratio']>0.1)|(temp_)]
# b['sku_id'] = [x for x in range(len(b))]
# b = b.astype(int)
# b.to_csv('pre_5_18_1.csv',index=False)

In [39]:
temp_b_2.describe()

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,cate_8,browse_count,cate_8_ratio
count,633.000000,633.0,633.0,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,...,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000,633.000000
mean,250114.148499,0.0,0.0,0.003160,0.004739,0.003160,0.004739,0.003160,0.001580,0.004739,...,0.244866,0.285940,0.255924,0.244866,0.417062,0.358610,6.740916,67.614534,242.955766,0.336980
std,29808.659328,0.0,0.0,0.056165,0.068734,0.056165,0.068734,0.056165,0.039746,0.068734,...,0.430348,0.452218,0.436725,0.430348,0.493463,0.479972,5.915479,137.256896,340.827053,0.273344
min,200077.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.004049
25%,225762.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,9.000000,48.000000,0.113744
50%,248681.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,24.000000,126.000000,0.250000
75%,275481.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,9.000000,67.000000,289.000000,0.523077
max,305258.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,46.000000,1917.000000,3300.000000,1.000000


# 5.17

In [17]:
# with open('./cache/Action_user_load_info_2016-02-01_2016-04-15','rb') as f:
#     user_load = pickle.load(f)
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# user_info_dataset = pd.read_csv(USER_FILE,encoding='gbk')
# user_info_dataset = pd.DataFrame(user_info_dataset[user_info_dataset['user_lv_cd']>=3]['user_id'].drop_duplicates())
# lv_bigger_3_user = pd.merge(user_info_dataset,user_dataset,on='user_id',how='left')
# user_load_browse_buy_frame = pd.merge(user_load,last_buy_cate_8_time,on='user_id',how='left')
# user_load_browse_buy_frame = pd.merge(user_load_browse_buy_frame,user_dataset_4,on='user_id',how='left')
# with open('./Action_user_load_browse_buy_time_frame','wb') as f:
#     pickle.dump(user_load_browse_buy_frame,f)
# with open('./cache/Action_user_load_browse_buy_time_frame','rb') as f:
#     user_load_browse_buy_frame = pickle.load(f)
# user_load_browse_buy_frame_1 = user_load_browse_buy_frame.drop(['cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'],axis=1)
# user_load_browse_buy_frame_1

,user_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,2016-02-09,...,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,last_buy_cate,last_buy_cate_8_time,cate_8,browse_count,cate_8_ratio
0,200001.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,12.0,2016-03-27 19:54:00,8,2016-03-27 19:54:00,107.0,203.0,0.527094
1,200003.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,13.0,2016-01-30 00:00:00,NaN,NaN,585.0,647.0,0.904173
2,200005.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,2016-02-23 23:00:00,NaN,NaN,92.0,120.0,0.766667
3,200007.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,8.0,2016-01-30 00:00:00,NaN,NaN,21.0,792.0,0.026515
4,200008.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,12.0,2016-01-30 00:00:00,NaN,NaN,141.0,311.0,0.453376
5,200010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,2016-01-30 00:00:00,NaN,NaN,90.0,90.0,1.000000
6,200011.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,2016-01-30 00:00:00,NaN,NaN,57.0,88.0,0.647727
7,200012.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,14.0,2016-01-30 00:00:00,NaN,NaN,141.0,190.0,0.742105
8,200013.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,15.0,2016-01-30 00:00:00,NaN,NaN,56.0,322.0,0.173913
9,200014.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,6.0,2016-04-05 10:47:00,NaN,NaN,7.0,165.0,0.042424


# 最后一次购买商品为第8类,最终没有一个用户在测试拿5天买了第8类商品

In [64]:
# temp_1 = user_load_browse_buy_frame_1[user_load_browse_buy_frame_1['last_buy_cate_8_time']==user_load_browse_buy_frame_1['last_buy_time']][['user_id','load_days','cate_8','browse_count','cate_8_ratio','last_buy_time']]
# temp_2 = temp_1[temp_1['load_days']<=3]     #1894
# # temp_3 = temp_2[(temp_2['cate_8_ratio']==1)|(temp_2['cate_8_ratio']==0)]    #1545
# # temp_4 = temp_2[(temp_2['cate_8_ratio']>0.815)&(temp_2['last_buy_time']>'2016-04-09 00:00:00')]
# temp_5 = temp_2[temp_2['last_buy_time']<'2016-04-09 00:00:00']
# temp_5 = temp_5.sort_values(by='cate_8',ascending=False)
# aa = pd.merge(temp_1,user_info_dataset,on='user_id',how='left')
# aa = aa[aa['user_lv_cd']>=3]
# bb = aa[(aa['age']!='46-55岁')]
# cc = pd.merge(bb,user_dataset,on='user_id',how='left')
# dd =cc
# dd = dd[dd['time']>dd['last_buy_time']]
# dd = dd[dd['cate']==8]
# dd = dd[dd['time']>'2016-04-09 00:00:00']
# dd = dd[dd['type']==2]
# # dd.describe()
# ee = pd.DataFrame(dd['user_id']).drop_duplicates()
# ee['sku_id'] = [x for x in range(len(ee))]
# ee = ee.astype(int)
# ee.to_csv('pre_5_17_3.csv',index=False)
# temp_6 = temp_1[(temp_1['cate_8_ratio']<1)&(temp_1['cate_8_ratio']>0)] 
# temp_6 = temp_6[temp_6['cate_8_ratio']>0.62]

In [77]:
temp_2[['cate_8','browse_count','cate_8_ratio']].describe()

,cate_8,browse_count,cate_8_ratio
count,1894.000000,1894.000000,1894.000000
mean,10.710665,14.216473,0.816193
std,20.728408,27.604666,0.341432
min,0.000000,1.000000,0.000000
25%,2.000000,3.000000,0.812500
50%,5.000000,6.000000,1.000000
75%,11.000000,15.000000,1.000000
max,376.000000,417.000000,1.000000


In [71]:
good_predect_area = user_load_browse_buy_frame_1[user_load_browse_buy_frame_1['last_buy_cate_8_time']==user_load_browse_buy_frame_1['last_buy_time']]
good_predect_area[['load_days','cate_8','browse_count','cate_8_ratio','last_buy_time']].describe()

,load_days,cate_8,browse_count,cate_8_ratio
count,9953.000000,9953.000000,9953.000000,9953.000000
mean,4.849392,76.305134,128.274490,0.716175
std,4.628081,149.029702,273.376847,0.319421
min,1.000000,0.000000,1.000000,0.000000
25%,2.000000,10.000000,16.000000,0.500000
50%,3.000000,31.000000,51.000000,0.850932
75%,6.000000,85.000000,138.000000,1.000000
max,44.000000,4535.000000,10756.000000,1.000000


# 5.16

In [57]:
# with open('./cache/Action_2016-02-01_2016-04-16_with_cate-time-type-user_id','rb') as f:
#     user_dataset = pickle.load(f)
# user_info_dataset = pd.read_csv(USER_FILE,encoding='gbk')
# #干掉用户等级低的，因为这群用户购买率非常低
# user_id = pd.DataFrame(user_dataset['user_id']).drop_duplicates()
# user_id_bigger_3 = pd.merge(user_id,user_info_dataset,on='user_id',how='left')
# user_id_bigger_3 = pd.DataFrame(user_id_bigger_3[user_id_bigger_3['user_lv_cd']>=3]['user_id'])
# user_dataset_1 = pd.merge(user_id_bigger_3,user_dataset,on='user_id',how='left')
# user_dataset_2 = user_dataset_1[user_dataset_1['type']==4]
# user_dataset_2 = user_dataset_2.groupby('user_id',as_index=False).time.max()
# user_dataset_2.columns = ['user_id','last_buy_time']
# user_dataset_3 = user_dataset_1[['user_id','time']]
# user_dataset_2 = pd.merge(user_dataset_3,user_dataset_2,on='user_id',how='left')
# user_dataset_2 = user_dataset_2.fillna('2016-01-30 00:00:00')
# user_dataset_2 = user_dataset_2[user_dataset_2['time']>user_dataset_2['last_buy_time']]
# user_dataset_4 = user_dataset_2[['user_id','time']]
# user_dataset_4['time'] = user_dataset_4.time.apply(lambda t:t[:10])
## user_dataset_4['last_buy_time'] = user_dataset_4.last_buy_time.apply(lambda t:t[:10])
# user_dataset_4 = user_dataset_4.drop_duplicates()
# user_dataset_4.to_csv('all_user_dataset.csv',index=False)
# type_dummies = pd.get_dummies(user_dataset_4['time'],prefix=None)
# user_load = pd.concat([user_dataset_4,type_dummies],axis=1)
# user_load = user_load.drop(['time'],axis=1)
# user_load = user_load.groupby('user_id',as_index=False).sum()
# user_load['load_days'] = user_load.sum(axis=1) - user_load['user_id']
# user_last_buy_data = user_dataset_2[['user_id','last_buy_time']].drop_duplicates()
# user_load = pd.merge(user_load,user_last_buy_data,on='user_id',how='left')
# with open('./cache/Action_user_load_info_2016-02-01_2016-04-15','wb') as f:
#     pickle.dump(user_load,f)
# user_dataset_1.groupby(['user_id','cate'],as_index=False).count()



#
# user_id = pd.DataFrame(user_dataset['user_id']).drop_duplicates()
# user_id_bigger_3 = pd.merge(user_id,user_info_dataset,on='user_id',how='left')
# user_id_bigger_3 = pd.DataFrame(user_id_bigger_3[user_id_bigger_3['user_lv_cd']>=3]['user_id'])
# user_dataset_1 = pd.merge(user_id_bigger_3,user_dataset,on='user_id',how='left')
# user_dataset_2 = user_dataset_1[user_dataset_1['type']==4]
# user_dataset_2 = user_dataset_2.groupby('user_id',as_index=False).time.max()
# user_dataset_2.columns = ['user_id','last_buy_time']
# user_dataset_3 = user_dataset_1[['user_id','time','cate']]
# user_dataset_2 = pd.merge(user_dataset_3,user_dataset_2,on='user_id',how='left')
# user_dataset_2 = user_dataset_2.fillna('2016-01-30 00:00:00')
# user_dataset_2 = user_dataset_2[user_dataset_2['time']>user_dataset_2['last_buy_time']]
# user_dataset_4 = user_dataset_2[['user_id','cate']]
# type_dummies = pd.get_dummies(user_dataset_4['cate'],prefix='cate')
# user_dataset_4 = pd.concat([user_dataset_4,type_dummies],axis=1)
# user_dataset_4 = user_dataset_4.drop(['cate'],axis=1)
# user_dataset_4 = user_dataset_4.groupby(['user_id'],as_index=False).sum()
# user_dataset_4['browse_count'] = user_dataset_4.sum(axis=1) - user_dataset_4['user_id']
# user_dataset_4['cate_8_ratio'] = user_dataset_4['cate_8']/user_dataset_4['browse_count']
# with open('./cache/Action_user_cate_ratio_2016-02-01_2016-04-15','wb') as f:
#     pickle.dump(user_dataset_4,f

#最后一次购买商品类型
# user_dataset_2 = user_dataset_1[user_dataset_1['type']==4]
# user_dataset_2 = user_dataset_2.groupby(['user_id','cate'],as_index=False).time.max()
# user_dataset_2.columns = ['user_id','last_buy_cate','last_buy_time']
# user_dataset_3 = user_dataset_1[['user_id','time']]
# user_dataset_2 = pd.merge(user_dataset_3,user_dataset_2,on='user_id',how='left')
# user_dataset_2['last_buy_time'] = user_dataset_2.last_buy_time.fillna('2016-01-30 00:00:00')
# user_dataset_2 = user_dataset_2[user_dataset_2['time']>user_dataset_2['last_buy_time']]
# user_dataset_2['last_buy_cate'] = user_dataset_2.last_buy_cate.replace('2016-01-30 00:00:00','NaN')
# last_buy_cate_user = user_dataset_2[['user_id','last_buy_cate','last_buy_time']].drop_duplicates()
# with open('./cache/Action_user_last_buy_cate','wb') as f:
#     pickle.dump(last_buy_cate_user,f)

In [18]:
with open('./cache/Action_user_load_info_2016-02-01_2016-04-15','rb') as f:
    user_load = pickle.load(f)
user_info_dataset = pd.read_csv(USER_FILE,encoding='gbk')
a = user_load[(user_load['load_days']<=3)&(user_load['last_buy_time']=='2016-01-30 00:00:00')]
b = a[(a['2016-04-15']==1)|(a['2016-04-14']==1)|(a['2016-04-13']==1)|(a['2016-04-12']==1)|(a['2016-04-11']==1)|(a['2016-04-10']==1)|(a['2016-04-09']==1)]
b = b[['user_id','2016-04-09','2016-04-10','2016-04-11','2016-04-12','2016-04-13','2016-04-14','2016-04-15','load_days','last_buy_time']]
b = pd.merge(b,user_info_dataset,on='user_id',how='left')
c = b[(b['age']=='-1')|(b['age']=='26-35岁')|(b['age']=='36-45岁')]
# c.sort_values(by='user_reg_tm',ascending=False)
user_id_b = pd.merge(c,user_dataset,on='user_id',how='left')
user_id_b

# temp = user_id_b[['user_id','cate','type']]
# temp.groupby(['user_id','cate'],as_index=False).count()

,user_id,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,age,sex,user_lv_cd,user_reg_tm,cate,time,type
0,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:53:49,1
1,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:53:55,1
2,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:54:45,1
3,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:54:46,6
4,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:54:58,1
5,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:54:59,6
6,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,6,2016-03-04 15:55:06,1
7,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,8,2016-03-04 15:55:13,1
8,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,8,2016-03-04 15:56:49,1
9,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,8,2016-03-04 15:56:50,6


In [19]:
d = user_id_b[(user_id_b['cate']==8)]
d = d[d['time']>'2016-04-09 00:00:00']
d_2 = pd.DataFrame(d[d['type']==2]['user_id'].drop_duplicates())
d_2 = d[d['type']==2].drop_duplicates()
# d_type_3 = d[d['type']==3]
# d_type_3

In [22]:
d[d['user_id']==214957]

,user_id,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,age,sex,user_lv_cd,user_reg_tm,cate,time,type
17335,214957.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2013-09-09,8,2016-04-10 10:41:03,2
17336,214957.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2013-09-09,8,2016-04-10 10:45:20,3
17337,214957.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2013-09-09,8,2016-04-10 10:45:20,3
17338,214957.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2013-09-09,8,2016-04-10 10:45:20,3


In [20]:
d_2

,user_id,2016-04-09,2016-04-10,2016-04-11,2016-04-12,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,age,sex,user_lv_cd,user_reg_tm,cate,time,type
26,200208.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,36-45岁,2.0,4,2013-04-11,8,2016-04-14 11:45:54,2
34,200288.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016-01-30 00:00:00,26-35岁,1.0,3,2016-01-29,8,2016-04-15 12:04:07,2
510,200756.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,2016-01-30 00:00:00,36-45岁,2.0,3,2016-02-01,8,2016-04-15 11:52:37,2
531,200756.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,3.0,2016-01-30 00:00:00,36-45岁,2.0,3,2016-02-01,8,2016-04-15 12:02:04,2
669,201123.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2016-02-04,8,2016-04-13 04:53:01,2
719,201123.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2016-02-04,8,2016-04-14 19:46:13,2
725,201123.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,2016-01-30 00:00:00,26-35岁,2.0,3,2016-02-04,8,2016-04-14 19:46:18,2
776,201409.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,2016-01-30 00:00:00,36-45岁,1.0,3,2010-08-15,8,2016-04-11 16:50:24,2
789,201409.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,3.0,2016-01-30 00:00:00,36-45岁,1.0,3,2010-08-15,8,2016-04-11 16:58:54,2
945,201568.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2016-01-30 00:00:00,36-45岁,2.0,4,2016-02-17,8,2016-04-15 16:54:38,2


In [40]:
# dd = pd.merge(d_2,d_type_3,on='user_id',how='left')
# dd['time'] = dd.time.fillna(0)
# dd1 = pd.DataFrame(dd[dd['time']==0]['user_id'].drop_duplicates())
# dd1

# name = pd.read_csv('a3.csv')
# ll = pd.merge(d_2,name,on='user_id',how='left')
# ll

# ll = pd.concat([dd,name],axis=0)['user_id']
# ll = pd.DataFrame(ll).drop_duplicates()
# ll['sku_id'] = [x for x in range(len(ll))]
# ll = pd.DataFrame(ll).astype(int)
# ll.to_csv('pre_9_17_2.csv',index=False)


In [41]:
ll

,user_id,sku_id
1569,247439,0
2508,218857,1
3256,223525,2
7235,277085,3
9076,248782,4
9740,247398,5
11666,237214,6
12941,288393,7
24732,298028,8
27258,207326,9


In [25]:
# ll_info = pd.me'cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'rge(ll,user_load_browse_buy_frame,on='user_id',how='left')
ll_info.drop(['cate_4','cate_5','cate_6','cate_7','cate_9','cate_10','cate_11'],axis=1)

,user_id,sku_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,...,2016-04-13,2016-04-14,2016-04-15,load_days,last_buy_time,last_buy_cate,last_buy_cate_8_time,cate_8,browse_count,cate_8_ratio
0,200208,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,2.0,2016-01-30 00:00:00,NaN,NaN,21.0,28.0,0.750000
1,200288,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,2016-01-30 00:00:00,NaN,NaN,136.0,140.0,0.971429
2,200756,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,2016-01-30 00:00:00,NaN,NaN,56.0,119.0,0.470588
3,201123,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,2.0,2016-01-30 00:00:00,NaN,NaN,92.0,92.0,1.000000
4,201409,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,3.0,2016-01-30 00:00:00,NaN,NaN,66.0,66.0,1.000000
5,201568,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,2016-01-30 00:00:00,NaN,NaN,3.0,3.0,1.000000
6,202845,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,3.0,2016-01-30 00:00:00,NaN,NaN,147.0,147.0,1.000000
7,203088,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2016-01-30 00:00:00,NaN,NaN,78.0,78.0,1.000000
8,203309,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,2016-01-30 00:00:00,NaN,NaN,22.0,26.0,0.846154
9,203327,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,2016-01-30 00:00:00,NaN,NaN,4.0,5.0,0.800000


In [27]:
ll_info.describe()

,user_id,sku_id,2016-02-01,2016-02-02,2016-02-03,2016-02-04,2016-02-05,2016-02-06,2016-02-07,2016-02-08,...,cate_4,cate_5,cate_6,cate_7,cate_8,cate_9,cate_10,cate_11,browse_count,cate_8_ratio
count,524.000000,524.000000,524.000000,524.000000,524.000000,524.0,524.000000,524.000000,524.0,524.0,...,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000,524.000000
mean,252744.227099,261.500000,0.007634,0.007634,0.003817,0.0,0.001908,0.003817,0.0,0.0,...,5.232824,3.667939,4.061069,4.188931,72.826336,2.185115,0.122137,0.185115,92.469466,0.852423
std,31683.791016,151.409348,0.087119,0.087119,0.061721,0.0,0.043685,0.061721,0.0,0.0,...,20.769683,14.570736,18.701338,23.699291,85.257626,14.717288,1.350719,1.024596,109.926836,0.254596
min,200208.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,3.000000,0.020690
25%,222790.750000,130.750000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,21.000000,0.000000,0.000000,0.000000,24.000000,0.823529
50%,251915.500000,261.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,43.500000,0.000000,0.000000,0.000000,59.000000,1.000000
75%,281911.500000,392.250000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,93.250000,0.000000,0.000000,0.000000,119.000000,1.000000
max,305287.000000,523.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,0.0,0.0,...,221.000000,159.000000,295.000000,297.000000,698.000000,233.000000,18.000000,10.000000,1105.000000,1.000000


In [41]:
# len(user_load)                                    #91581
# len(user_load[user_load['load_days']<=45])        #91133
# len(user_load[user_load['load_days']<=40])        #90711
# len(user_load[user_load['load_days']<=35])        #90017
# len(user_load[user_load['load_days']<=30])        #88854
# len(user_load[user_load['load_days']<=25])        #86522
# len(user_load[user_load['load_days']<=20])        #82220
# len(user_load[user_load['load_days']<=15])        #73600
# len(user_load[user_load['load_days']<=10])        #5752l 
# len(user_load[user_load['load_days']<=9])         #52382
# len(user_load[user_load['load_days']<=8])         #47247
# len(user_load[user_load['load_days']<=7])         #41549
# len(user_load[user_load['load_days']<=6])         #35554
# len(user_load[user_load['load_days']<=5])         #29323 
# len(user_load[user_load['load_days']<=4])         #22737
# len(user_load[user_load['load_days']<=3])         #16511
# len(user_load[user_load['load_days']<=2])         #10600
# len(user_load[user_load['load_days']<=1])         #4538

In [55]:
len(user_id_c[(user_id_c['cate']==8)&((user_id_c['type']==2)&(user_id_c['type']!=3))]['user_id'].drop_duplicates())      #541

131